In [ ]:
!pip install selenium
!pip install pyautogui
!pip install openpyxl

In [11]:
import re
import time
import locale
import pyautogui
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import openpyxl

def find_and_send_keys(id, key):
    time.sleep(0.5)
    element = wait.until(EC.presence_of_element_located((By.ID, id)))
    element.send_keys(key)

def change_cell_background(sheet, cell_range, color):
    for row in sheet.iter_rows(min_row=cell_range[0], max_row=cell_range[1], min_col=cell_range[2], max_col=cell_range[3]):
        for cell in row:
            cell.fill = openpyxl.styles.PatternFill(start_color=color, end_color=color, fill_type="solid")

locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')
start = datetime.now()
format_date_time = "%B %Y"

# Archivo Excel local
excel_file_path = f'./CUFE NOKIA {start.strftime(format_date_time).upper()}.xlsx'
cufe_column_index = 2  # Columna con los CUFEs
hoja_nombre = "CORTE DIAN"  # Nombre de la hoja
wb = openpyxl.load_workbook(excel_file_path)
ws = wb[hoja_nombre]

# Leer toda la columna de CUFEs y almacenarla en una lista
cufe_list = [str(ws.cell(row=row_num, column=cufe_column_index).value) for row_num in range(2, ws.max_row + 1)]
print("CUFE's cargados")
#wb.close()
# Lista para almacenar resultados True/False
result_list = []

driver = webdriver.Chrome()
wait = WebDriverWait(driver, 20)

for cufe in cufe_list:
    if cufe == "None":
        break
    driver.get('https://catalogo-vpfe.dian.gov.co/User/SearchDocument')
    time.sleep(0.5)
    find_and_send_keys("DocumentKey", cufe)
    time.sleep(0.5)
    pyautogui.press('enter')
    time.sleep(0.2)
    while True:
        page_source = driver.page_source
        if re.search(r"Recaptcha inválido.", page_source):
            time.sleep(1)
            pyautogui.press('enter')
        else:
            break
    time.sleep(0.3)
    page_source = driver.page_source

    if re.search(r"Aceptación expresa de la Factura Electrónica de Venta", page_source):
        #print(f"El texto se encontró para el CUFE {cufe}.")
        result_list.append(True)
    else:
        #print(f"El texto NO se encontró para el CUFE {cufe}.")
        result_list.append(False)
driver.quit()
print("Búsqueda finalizada")
columna_q = ws['Q'][1:]
for valor_booleano, celda in zip(result_list, columna_q):
    celda.value = valor_booleano
wb.save("CUFE NOKIA {start.strftime(format_date_time).upper()}.xlsx")
wb.close()
print("Registro finalizado")

CUFE's cargados
Búsqueda finalizada
Registro finalizado
